In [21]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import os
import warnings
import unidecode

warnings.filterwarnings("ignore")

# Cargar datasets preprocesados
df_completo = pd.read_parquet(r'../Datasets/dataset_completo.parquet')

In [22]:
# Crear la matriz de características basada en las columnas relevantes
def generar_features_matrix(df):
    features = df[['vote_average', 'vote_count', 'popularity']].fillna(0)
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)
    return features_scaled

# Generar la matriz de características en memoria
features_matrix = generar_features_matrix(df_completo)

# Crear una columna de índice para asegurar que se pueda mapear correctamente
df_completo = df_completo.reset_index(drop=True)

In [23]:
# Función de recomendación basada en la similitud del coseno
def recomendacion(titulo, n_recomendaciones=5):
    try:
        # Limpiar y normalizar el título de la película
        titulo_normalizado = unidecode.unidecode(titulo.lower().strip())

        # Buscar la película en el DataFrame
        pelicula_idx = df_completo[df_completo['title'].str.contains(titulo_normalizado, case=False, na=False)].index

        if len(pelicula_idx) == 0:
            return {"mensaje": f"El título '{titulo}' no se encuentra en el dataset."}
        
        # Obtener el índice de la película en el DataFrame
        idx = pelicula_idx[0]

        # Asegurarse de que el índice esté dentro de los límites de la matriz de características
        if idx >= len(features_matrix):
            return {"mensaje": "Error: el índice de la película está fuera de los límites del conjunto de datos."}

        # Calcular las similitudes utilizando la matriz de características
        cosine_similarities = cosine_similarity([features_matrix[idx]], features_matrix).flatten()

        # Verificar que el número de recomendaciones sea válido
        n_recomendaciones = min(n_recomendaciones, len(cosine_similarities) - 1)

        # Obtener los índices de las películas más similares, excluyendo la película original
        similar_indices = cosine_similarities.argsort()[::-1][1:n_recomendaciones + 1]

        # Verificar que los índices de recomendaciones están dentro del rango de filas del DataFrame
        similar_indices = [i for i in similar_indices if i < len(df_completo)]

        # Obtener los títulos de las películas recomendadas
        recomendaciones = df_completo['title'].iloc[similar_indices].tolist()

        return {"recomendaciones": recomendaciones}

    except Exception as e:
        return {"mensaje": f"Error interno en la recomendación: {str(e)}"}

In [24]:
# Probar la función de recomendación
resultado = recomendacion("twilight")
print(resultado)

{'recomendaciones': ["the huntsman winter's war", 'the last airbender', 'couples retreat', 'spy kids 3-d game over', "what's eating gilbert grape"]}
